In [1]:
import json
 
import pandas as pd
from tqdm import tqdm

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_original_graph/1989_stage2_400/verified_conversations.jsonl'
graph_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_original_graph/1989_stage2_400/verified_graphs.jsonl'    
conv = pd.read_json(conv_path, lines=True)
graph = pd.read_json(graph_path, lines=True)

In [3]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/ablation_study/fine_tune_text_LLM_with_original_graph/1989_stage2_400/conversations.jsonl'

In [4]:
sample_index = [782, 142, 631, 1223, 769, 847, 1943, 1322, 1021, 1794, 1749, 188, 1192, 1414, 585, 823, 340, 453, 949, 751, 390, 649, 763, 639, 280, 468, 421, 1743, 1175, 836, 1245, 122, 739, 1452, 50, 787, 686, 1438, 1317, 750, 3, 76, 1648, 876, 403, 1564, 1839, 14, 210, 12, 8, 18, 674, 1721, 1496, 90, 1681]

In [5]:
conv.drop(sample_index, inplace=True)
graph.drop(sample_index, inplace=True)
    


In [7]:
conv = conv.reset_index(drop=True)
graph = graph.reset_index(drop=True)

In [6]:
len(conv), len(graph)

(1932, 1932)

In [7]:
conv.iloc[0]

Instruction    Implement a modular Arithmetic Logic Unit (ALU...
Response       module alu(\n    input clk,\n    input rst,\n ...
task_id                                                       pe
Name: 0, dtype: object

In [8]:
graph.iloc[0]

type                                                      module_graph
edge_attrs                                                          []
has_edge_features                                                False
nodes                [{'id': 0, 'content': 'clk', 'type': 'input po...
connectivity         [[2, 3, 2, 3, 2, 3, 2, 3, 7, 9, 8, 10], [7, 7,...
Name: 0, dtype: object

In [9]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

In [10]:
system_prompt = 'Please act as a professional Verilog designer. Given a submodules interconnection graph:  \n'
for i in tqdm(range(len(conv))):
    new_conversations = {}
    new_graph = "nodes: {}, \nedge_attrs: {}, \nconnectivity: {}".format(graph['nodes'][i], graph['edge_attrs'][i], graph['connectivity'][i])
    text_graph = "\n{}\n".format(new_graph)
    new_instructions = system_prompt + text_graph + conv.iloc[i]['Instruction']
    new_instructions = new_instructions + "\n" + find_module_string(conv.iloc[i]['Response'])
    new_conversations['Instruction'] = new_instructions
    new_conversations['Response'] = conv.iloc[i]['Response']
    new_conversations['task_id'] = conv.iloc[i]['task_id']
    with open(new_conv_path, 'a') as f:
        json.dump(new_conversations, f)
        f.write('\n')



  0%|          | 0/1932 [00:00<?, ?it/s]

100%|██████████| 1932/1932 [00:00<00:00, 3202.42it/s]


In [11]:
conv

,Instruction,Response,task_id
0,Implement a modular Arithmetic Logic Unit (ALU...,"module alu(\n input clk,\n input rst,\n ...",pe
1,Design a Verilog module for a binary up/down c...,"module binary_counter(\n input clk,\n in...",square_wave
2,Design a modular Verilog setup for a 32-bit Ba...,"module shifter(\n input clk,\n input rst...",pe
3,Design a Verilog module to compute the bitwise...,"module xor_8bit(\n input [7:0] a, b,\n o...",adder_8bit
4,Design a Verilog module for a dual-channel Pul...,"module dual_pwm_generator(\n input clk,\n ...",systolic1x4
...,...,...,...
1927,Design a 32-bit arithmetic logic unit (ALU) th...,"module modular_ALU (\n input wire clk,\n ...",up_down_counter
1928,Design a Verilog module named `binary_adder_su...,module binary_adder_subtractor (\n input wi...,up_down_counter
1929,Implement a module for a 4-bit arithmetic logi...,`define ADD_OP 2'b00\n`define SUB_OP 2'b01\n`d...,stagepipe3
1930,Design and implement a Verilog module for a 32...,"module alu_32bit(\n input [1:0] op_code, //...",comparator_32bit
